# Import Libraries

<div class="alert alert-success">
<h2> Комментарий</h2>

Постарайся, чтобы имена столбцов у тебя были такие же, как и мои. 
У меня если в столбце более двух слов, я их разделял нижним подчеркиванием.
Я сохраню таблицы в csv, которые я в самом начале буду загружать из базы данных SQL, чтобы тебе не переделывать у себя всё.

Посмотри пункт про тестирование гипотез и Fundamentals Analisys. Может ещё что нибудь придумаешь

</div>

In [1]:
!pip install arch

^C
Traceback (most recent call last):
  File "/Users/vladislav/PycharmProjects/pythonProject/venv/bin/pip", line 8, in <module>
    sys.exit(main())
  File "/Users/vladislav/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/pip/_internal/cli/main.py", line 73, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/Users/vladislav/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/pip/_internal/commands/__init__.py", line 105, in create_command
    module = importlib.import_module(module_path)
  File "/Users/vladislav/.pyenv/versions/3.8.6/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import gmean
from IPython.display import Image
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
import arch
from sklearn.model_selection import GridSearchCV
from scipy import stats as st
import warnings
warnings.simplefilter('ignore')
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import pymssql


'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) '''

ImportError: dlopen(/Users/vladislav/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pymssql.cpython-38-darwin.so, 2): Library not loaded: /usr/local/opt/freetds/lib/libsybdb.5.dylib
  Referenced from: /Users/vladislav/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pymssql.cpython-38-darwin.so
  Reason: image not found

In [ ]:

conn = pymssql.connect(server='localhost', 
                       user='sa', 
                       password='YourPassword123', 
                       database='master')

# Import data from SQL DB to Pandas

In [ ]:
sec_df = pd.read_sql('SELECT * FROM securities', conn)
price_df = pd.read_sql('SELECT * FROM prices', conn)
fund_df = pd.read_sql('SELECT * FROM fundamentals_extracted_', conn)

In [ ]:
price_df = price_df.drop('Idx', axis=1)

In [ ]:
price_df_merged = pd.read_sql('SELECT * FROM prices INNER JOIN securities on prices.symbol = securities.Ticker_symbol INNER JOIN  fundamentals_extracted_ on fundamentals_extracted_.Ticker_Symbol = prices.symbol AND fundamentals_extracted_.For_Year = prices.for_year', conn)


In [ ]:
'''
# Здесь код чтобы сохранить загруженные из SQL таблички.
sec_df.to_csv('my_securities.csv')
price_df.to_csv('my_prices.csv')
fund_df.to_csv('my_fundamentals.csv')
'''

In [ ]:
fund_df_merged = pd.read_sql('SELECT * FROM fundamentals_extracted_ INNER JOIN securities on fundamentals_extracted_.Ticker_Symbol = securities.Ticker_symbol', conn)


# Preview data

In [ ]:
price_df.sample(6)

In [ ]:
price_df.info()

In [ ]:
sec_df.sample(6)


In [ ]:
sec_df.info()

In [ ]:
fund_df.sample(6)

In [ ]:
fund_df.info()

# Data Preprocessing and Cleaning

Удалим ненужные столбцы

In [ ]:
price_df = price_df.drop('ID', axis=1)
fund_df = fund_df.drop('to_drop', axis=1)
fund_df = fund_df.drop('ID', axis=1)
fund_df_merged = fund_df_merged.drop('to_drop', axis=1)
fund_df_merged = fund_df_merged.drop('ID', axis=1)
sec_df = sec_df.drop('ID', axis=1)

Приведем даты к нужному формату

In [ ]:
price_df['date'] = pd.to_datetime(price_df['date'])
sec_df['Date_first_added'] = pd.to_datetime(sec_df['Date_first_added'])

Проверим, есть ли пропуски

In [ ]:
price_df.isna().sum()

In [ ]:
sec_df.isna().sum()

Есть пропуски только в столбце, обозначающем дату включения в лист Нью-Йоркской Фондовой Биржи, этот столбец не не сёт особой смысловой нагрузки для нас

In [ ]:
fund_df.isna().sum()

# Exploratory Data Analysis

## Delete inconsistent companies

Проблема в котировках заключается в том, что котировки у разных компаний начинаются с разных дат (и даже разных годов). Нужно выбрать только те компании, по которым доступны котировки с 2012-01-18. 

In [ ]:
fund_df['For_Year'].value_counts()

In [ ]:
len(fund_df['Ticker_Symbol'].unique())

Всего 360 уникальных компаний, по которым доступны данные фундаментальных характеристик.

In [ ]:
len(fund_df[fund_df['For_Year'] == 2012]['Ticker_Symbol'].unique())

Из них половина доступна с 2012 года

In [ ]:
pd.Series(fund_df[fund_df['For_Year'] == 2012]['Ticker_Symbol'].unique())\
          .isin(price_df[price_df['date'] == pd.to_datetime('2012-01-18')]['symbol'].unique()).sum()

При этом по 178 из них имеются котировки с самого начала 2012 года.

In [ ]:
pd.Series(fund_df[fund_df['For_Year'] == 2012]['Ticker_Symbol'].unique()).isin(sec_df['Ticker_symbol'].unique()).sum()


С данными из SEC по этим компаниям проблем нет

Оставим только 178 указанных компаний в выборке для анализа, так как по остальным недостаточно данных:

In [ ]:
tickers_chosen = fund_df[fund_df['For_Year'] == 2012]['Ticker_Symbol'].unique()
tickers_chosen = tickers_chosen[pd.Series(fund_df[fund_df['For_Year'] == 2012]['Ticker_Symbol'].unique())\
          .isin(price_df[price_df['date'] == pd.to_datetime('2012-01-18')]['symbol'].unique())]
len(tickers_chosen)

In [ ]:
np.shape(fund_df)

In [ ]:
fund_df = fund_df[fund_df['Ticker_Symbol'].isin(tickers_chosen)]
fund_df_merged = fund_df_merged[fund_df_merged['Ticker_Symbol'].isin(tickers_chosen)]
sec_df = sec_df[sec_df['Ticker_symbol'].isin(tickers_chosen)]
price_df = price_df[price_df['symbol'].isin(tickers_chosen)]

In [ ]:
np.shape(fund_df)

Создадим pivot_table, в которой каждый столбец будет соответствовать отдельной компании, и в нем будет содержаться временной ряд для ежедневных доходностей этой компании.

In [ ]:
price_df

In [ ]:
sec_df.replace(['Telecommunications Services'], ['Telecommunications'], inplace=True)

sec_df = sec_df.rename(columns = {'Ticker_symbol' : 'symbol','GICS_Sector' : 'sector'})
price_df  = price_df.merge(sec_df[['symbol','sector']], on = 'symbol')
#price_df  = price_df.merge(fund_df_merged['Estimated Shares Outstanding'], left_on = ['symbol', 'year'], right_on = ['Ticker_Symbol', 'For_Year'])
price_df['date'] = pd.to_datetime(price_df['date'])

date_chosen = '2012-01-18'
price_df = price_df[pd.to_datetime(price_df['date']) > pd.to_datetime(date_chosen)]

sector_pivot = pd.pivot_table(price_df, values = 'close', index = ['date'], columns = ['sector']).reset_index()

price_df['return'] = price_df.close / price_df.close.shift(1)
price_df['good'] = price_df['symbol'] == price_df['symbol'].shift(1)
price_df = price_df.drop(price_df[price_df['good'] == False].index)
price_df.dropna(inplace = True)

price_df_copy = price_df.copy()

price_df.set_index(['symbol', 'date'], inplace=True, drop=False)

price_df['num shares outstanding'] = 0

columns_list = price_df.columns

#fund_df_merged = fund_df_merged.rename(columns = {'symbol' : 'Ticker_Symbol','sector' : 'sectorGICS_Sector', 'year': 'For_Year'})


In [ ]:
port_df = price_df.pivot('date', 'symbol', 'return').fillna(1)
port_df

In [ ]:
port_df.tail()

In [ ]:
print(port_df.columns.unique())

Возьмем котировки компании "Chevron" и 3 случайные компании и построим небольшую визуализацию:

In [ ]:
sample_tickers = pd.Series('CVX')
sample_tickers = sample_tickers.append(pd.Series(port_df.columns.unique()).sample(3))

fig, axs = plt.subplots(2, 1, figsize=(8, 10))

sns.lineplot(data=price_df.pivot('date', 'symbol', 'close')[sample_tickers].rolling(4).mean(), 
             palette="Set2", 
             linewidth=2.5, 
             ax=axs[0])
sns.violinplot(data=port_df[sample_tickers]-1, 
               palette="Set2", 
               inner="points", 
               orient="h", 
               scale='area', 
               ax=axs[1])

In [ ]:
plt.close()

## Fundamentals Analysis

Здесь используем неизмененный датасет fund_df_merged:

In [ ]:
fund_df_merged.sample(5)

In [ ]:
fund_df_merged.columns

In [ ]:
fund_df_merged['GICS_Sector'].value_counts()

In [ ]:
def delete_outliers(series):
    q1 = series.quantile(0.25)                 
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    iqr_test = (series >= (q1 - 1.5 * iqr)) & (series <= (q3 + 1.5 * iqr))
    return series.loc[iqr_test]

!pip install seaborn --upgrade

### Средние доходности по секторам

In [ ]:
sector_df_mean = pd.DataFrame()
sector_df_std = pd.DataFrame()
for year in [2012, 2013, 2014, 2015]:
    sector_df_mean[year] = (price_df.query('for_year == @year').groupby('sector')['return'].apply(lambda x: gmean(x)) - 1) * 252
    sector_df_std[year] =  price_df.query('for_year == @year').groupby('sector')['return'].std()
sector_df_mean.columns = sector_df_mean.columns.astype('str')
sector_df_std.columns = sector_df_std.columns.astype('str')

In [ ]:
sector_df_mean

In [ ]:
sector_df_std

График доступен в интерактивном виде в Дашборде

In [ ]:
px.bar(sector_df_mean, x=sector_df_mean.index, y=sector_df_mean['2015'], color=sector_df_mean.index, title='Средние доходности акций по отраслям в 2015г.')

### Рентабельность собственного капитала

In [ ]:
cols = ['After_Tax_ROE']
sectors = fund_df_merged['GICS_Sector'].unique()
fund_df_merged_del = fund_df_merged.query('GICS_Sector in @sectors')
fund_df_merged_del[cols] = fund_df_merged_del[cols].apply(delete_outliers)

f, ax = plt.subplots(figsize=(5, 13))

#sns.histplot(fund_df_merged_prcd, x='After_Tax_ROE', hue='GICS_Sector', element="step", stat="density")

sns.violinplot(data=fund_df_merged_del, x='After_Tax_ROE', 
               y=fund_df_merged_del['GICS_Sector'], 
               linewidth=1, 
               scale='area', 
               ax=ax, orient="h")
sns.despine(left=True)

plt.title('After Tax ROE by Sector', fontsize='xx-large')

### Валовая и чистая рентабельность по секторам

In [ ]:
sns.set_theme(style="whitegrid")
year = 2015
cols = ['Profit_Margin', 'Gross_Margin']
sectors = ["Industrials", "Information Tecknology", "Energy", "Materials", "Financials", 'Consumer Discretionary']
fund_df_merged_prcd = fund_df_merged.query('GICS_Sector in @sectors')
fund_df_merged_prcd[cols] = fund_df_merged_prcd[cols].apply(delete_outliers)

f, ax = plt.subplots(figsize=(8, 8))

sns.despine(f, left=True, bottom=True)
clarity_ranking = fund_df_merged['GICS_Sector'].unique()
sns.scatterplot(x='Gross_Margin', y='Profit_Margin',
                hue=fund_df_merged_prcd['GICS_Sector'],
                palette="CMRmap",
                linewidth=0, sizes=(1, 8),
                data=fund_df_merged_prcd.query('For_Year == @year'), ax=ax)
plt.title('Gross Margin vs Profit Margin by sector', fontsize='xx-large')

In [ ]:
sectors = fund_df_merged['GICS_Sector'].unique()

sns.jointplot(x=fund_df_merged.query('GICS_Sector in @sectors and Profit_Margin<=40')['Gross_Margin'], 
              y=fund_df_merged.query('GICS_Sector in @sectors and Profit_Margin<=40')['Profit_Margin'], 
              kind="hex", color="SlateBlue")


In [ ]:
sectors = fund_df_merged['GICS_Sector'].unique()
sns.jointplot(x=fund_df_merged.query('GICS_Sector in @sectors and Quick_Ratio<=400')['Cash_Ratio'], 
              y=fund_df_merged.query('GICS_Sector in @sectors and Quick_Ratio<=400')['Quick_Ratio'], 
              kind="hex", color="plum")

### Капитализация по отраслям

In [ ]:
price_df_merged['Capitalization'] = price_df_merged['close'] * price_df_merged['Estimated Shares Outstanding']

In [ ]:
capitalizations_sector =  price_df_merged.pivot_table(index='date', columns='GICS_Sector', values='Capitalization', aggfunc='sum').fillna(0)
capitalizations_sector['sum'] = 1
for sector in capitalizations_sector.columns:
    if sector != 'sum':
        capitalizations_sector['sum'] += capitalizations_sector[sector]
capitalizations_sector_copy = capitalizations_sector.drop('sum', axis=1).copy()
for sector in capitalizations_sector.columns:
    capitalizations_sector[sector] = capitalizations_sector[sector]/capitalizations_sector['sum']
capitalizations_sector.drop('sum', axis=1, inplace=True)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 6))
sns.lineplot(data=capitalizations_sector, 
             palette="Set3", 
             linewidth=2.5, ax=axs)

capitalizations_stock = price_df_merged.pivot_table(index='date', columns='symbol', values='Capitalization').fillna(0)
capitalizations_sector = price_df_merged.pivot_table(index='date', columns='GICS_Sector', values='Capitalization', aggfunc='sum').fillna(0)
sector_list = capitalizations_sector.columns
stock_list = capitalizations_stock.columns
for sector in tqdm(sector_list):
    for stock in stock_list:
        if stock in price_df_merged.query('GICS_Sector==@sector')['symbol'].unique():
            capitalizations_stock[stock] = capitalizations_stock[stock] / capitalizations_sector[sector]
            

capitalizations_stock = capitalizations_stock.loc[capitalizations_stock.index >= pd.to_datetime('2012-01-19')]
capitalizations_stock = capitalizations_stock[price_df.pivot('date', 'symbol', 'return').columns]
capitalizations_stock

for sector in tqdm(capitalizations_sector.columns):
    port_df[sector] = 1
    s = str(sector) + '_cumulative'
    port_df[s] = 1
    stock_list = price_df.query('sector == @sector')['symbol'].unique()
    for col in port_df.columns:
        if col in stock_list:
            port_df[sector] *= (port_df[col] * capitalizations_stock[stock])
    port_df[sector].plot()

## Hypotheses Testing

Посмотрим, какие вообще секторы есть

In [ ]:
fund_df_merged['GICS_Sector'].value_counts()

### Гипотеза о равенстве средних Profit Margin для секторов Energy и Materials

Посмотрим, сколько уникальных компаний принадлежат секторам Materials и Energy

In [ ]:
len(fund_df_merged.query('For_Year == 2015 and GICS_Sector != "Energy"')['Ticker_Symbol'].unique())

In [ ]:
len(fund_df_merged.query('For_Year == 2015 and GICS_Sector == "Energy"')['Ticker_Symbol'].unique())

Проверим предположение о том, что их Profit Margin в 2015 равны:
* H0: Profit Margin для компаний секторов Energy и Materials в 2015 в среднем равны (так как компании из обоих секторов занимаются добычей полезных ископаемых)
* H1: Profit Margin для компаний секторов Energy и Materials в 2015 в среднем не равны

In [ ]:
sector_1 = "Energy"
sector_2 = "Materials"
column = 'Profit_Margin'
alpha = 0.05
print('Уровень значимости: ', alpha)
print()
for year in [2013, 2014, 2015]:
    sample_1 = fund_df_merged.query('GICS_Sector == @sector_1 and For_Year == @year')[column]
    sample_2 = fund_df_merged.query('GICS_Sector == @sector_2 and For_Year == @year')[column]
    # без удаления выбросов:
    #results = st.ttest_ind(sample_1, sample_2, equal_var=False)
    # с удалением выбросов:
    results = st.ttest_ind(delete_outliers(sample_1), delete_outliers(sample_2), equal_var=False)
    print(year, ' год : p-значение: ', results.pvalue)

    if (results.pvalue < alpha):
        print("Отвергаем нулевую гипотезу")
        print()
    else:
        print("Не получилось отвергнуть нулевую гипотезу")
        print()

### Гипотеза: средняя доходность акций компаний сектора Energy меньше, чем в средняя доходность по другим отраслям

Проверим данное предположение:
* H0:  средняя доходность акций компаний сектора Energy равна средней доходности по другим отраслям
* H1:  средняя доходность акций компаний сектора Energy меньше, чем в средняя доходность по другим отраслям

In [ ]:
capitalizations_sector_copy.head(3)

In [ ]:
capitalizations_sector_copy.drop(sector_1, axis=1).columns

In [ ]:
sector_1 = "Energy"
alpha = 0.05
print('Уровень значимости: ', alpha)
print()
sample_1 = capitalizations_sector_copy[sector_1]
sample_2 = capitalizations_sector_copy[capitalizations_sector_copy.drop(sector_1, axis=1).columns].mean(axis=1)
# без удаления выбросов:
#results = st.ttest_ind(sample_1, sample_2, equal_var=False)
# с удалением выбросов:
results = st.ttest_ind(delete_outliers(sample_1), delete_outliers(sample_2), equal_var=False)
print('p-значение: ', results.pvalue)
if (results.pvalue < alpha/2):
    print("Отвергаем нулевую гипотезу")
    print()
else:
    print("Не получилось отвергнуть нулевую гипотезу")
    print()

### Ещё какую-нибудь гипотезу с ликвидностью сделай по аналогии с первой гипотезой

# GARCH Model

In [ ]:
port_df.columns[20:100]

## ACF- and PACF-function

In [ ]:
stock = 'CVX'  #CVX EIX DHR DUK COG
lags = 50
pq_1 = [10, 10]
pq_2 = [5, 5]

plot_acf(port_df[stock], lags=lags, zero=False)

In [ ]:
plot_pacf(port_df[stock], lags=lags, zero=False)

## ADF test

* H0: there is a unit root
* H1: there is no unit root

In [ ]:
n_test = 15
port_df_stock = ((port_df[stock]-1)*100)[16:]
port_df_stock_std = ((port_df[stock]-1)*100).rolling(15).std()[16:]
train, test = port_df_stock[:-n_test], port_df_stock[-n_test:]

In [ ]:
adfuller(port_df_stock)

P-value < 0.05   =>   Отвергаем нулевую гипотезу H0

## Building GARCH Model

In [ ]:
port_df_stock.plot()
plt.title('Standard Deviation of ' + stock)

In [ ]:
model_1 = arch.arch_model(np.ascontiguousarray(train-1), mean='Zero', vol='GARCH', p=pq_1[0], o=0 ,q=pq_1[1])
model_fit_1 = model_1.fit()
y_pred_1 = model_fit_1.forecast(horizon=n_test)

model_2 = arch.arch_model(np.ascontiguousarray(train-1), mean='Zero', vol='GARCH', p=pq_2[0], o=0, q=pq_2[1])
model_fit_2 = model_2.fit()
y_pred_2 = model_fit_2.forecast(horizon=n_test)

In [ ]:
model_fit_1.summary()

In [ ]:
model_fit_2.summary()

In [ ]:
model_fit_1.plot()

In [ ]:
model_fit_2.plot()

In [ ]:
test_std_1 = pd.DataFrame({'actual': port_df_stock_std[-n_test:], 'predicted': np.sqrt(y_pred_1.variance.values[-1, :])})
test_std_2 = pd.DataFrame({'actual': port_df_stock_std[-n_test:], 'predicted': np.sqrt(y_pred_2.variance.values[-1, :])})

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 10))
sns.lineplot(data=test_std_1, 
             palette="Set2", 
             linewidth=2.5, 
             ax=axs[0])
axs[0].set_title('Model 1')
sns.lineplot(data=test_std_2, 
             palette="Set2", 
             linewidth=2.5, 
             ax=axs[1])
axs[1].set_title('Model 2')

# Dash

In [ ]:
!pip install jupyter-dash

In [ ]:
sector_df_mean = pd.DataFrame()
sector_df_std = pd.DataFrame()
for year in [2012, 2013, 2014, 2015]:
    sector_df_mean[year] = (price_df.query('for_year == @year').groupby('sector')['return'].apply(lambda x: gmean(x)) - 1) * 252
    sector_df_std[year] =  price_df.query('for_year == @year').groupby('sector')['return'].std()
sector_df_mean.columns = sector_df_mean.columns.astype('str')
sector_df_std.columns = sector_df_std.columns.astype('str')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

stock = 'CVX'
sector = 'Energy'

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

#fig = px.line(price_df, x=price_df.pivot('date', 'symbol', 'close').index, y=price_df.pivot('date', 'symbol', 'close')['CVX'], title='Chevron Corp. Stock Prices')

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(x=price_df.pivot('date', 'symbol', 'close').index, 
                            y=price_df.pivot('date', 'symbol', 'close')[stock], mode = 'lines',
                            name = 'Chevron Corp. Stock Prices'), 
                 row=1, col=1)

fig.append_trace(go.Scatter(
    x=[2, 3, 4],
    y=[100, 110, 120],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[0, 1, 2],
    y=[10, 11, 12]
), row=3, col=1)




fig.update_layout(height=1400, width=1400, title_text="Stacked Subplots")

# app appearance
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),
    
    html.Div(children='''
        Fuck you Lakshina!
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    ),
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} for x in sector_df_mean.columns],
        value= sector_df_mean.columns[0],
        clearable=False,
        placeholder="Select a year",
    ),
    dcc.Graph(id="bar-chart")
])

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(value):
    fig = px.bar(sector_df_mean, x=sector_df_mean.index, y=sector_df_mean[value], color=sector_df_mean.index, title='Средние доходности акций по отраслям за '+str(value)+' год')
    return fig

if __name__ == '__main__':
    app.run_server() # NOT WORK!

One of: ['area', 'bar', 'barpolar', 'box',
                     'candlestick', 'carpet', 'choropleth',
                     'choroplethmapbox', 'cone', 'contour',
                     'contourcarpet', 'densitymapbox', 'funnel',
                     'funnelarea', 'heatmap', 'heatmapgl',
                     'histogram', 'histogram2d',
                     'histogram2dcontour', 'image', 'indicator',
                     'isosurface', 'mesh3d', 'ohlc', 'parcats',
                     'parcoords', 'pie', 'pointcloud', 'sankey',
                     'scatter', 'scatter3d', 'scattercarpet',
                     'scattergeo', 'scattergl', 'scattermapbox',
                     'scatterpolar', 'scatterpolargl',
                     'scatterternary', 'splom', 'streamtube',
                     'sunburst', 'surface', 'table', 'treemap',
                     'violin', 'volume', 'waterfall']